In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from splot.mapping import vba_choropleth, vba_legend, mapclassify_bin
from shapely.geometry import Point, Polygon, LineString
import libpysal.weights as sw 
from esda.moran import Moran_Local, Moran
from splot.esda import lisa_cluster, plot_local_autocorrelation, plot_moran
import warnings
import census
import us
warnings.simplefilter("ignore") 

In [2]:
c = census.Census('bbf0f9cd4d3e9a9bb4c295578e2e6cb2765ced0d', year=2020)

In [23]:
#test
c.acs5.state(('NAME', 'B19013_001E'), us.states.MD.fips)

[{'NAME': 'Maryland', 'B19013_001E': 87063.0, 'state': '24'}]

In [29]:
#access census data
#variable names: total, white alone, black alone, asian alone, median household income
#consider ratio of income to poverty level? B17002
acs_list = c.acs5.state_county_tract(('NAME', 'B02001_001E','B02001_002E', 'B02001_002E', 'B02001_005E','B19013_001E'), us.states.NY.fips, '005', census.ALL)

In [26]:
nyc_acs = pd.DataFrame(acs_list)

In [37]:
codes={'manhattan':'061', 'staten_island':'085','bronx':'005', 'queens':'081', 'brooklyn':'047'}
list_of_lists=[]
for i in codes:
    acs_list = c.acs5.state_county_tract(('NAME', 'B02001_001E','B02001_002E', 'B02001_003E', 'B02001_005E','B19013_001E'), us.states.NY.fips, codes[i], census.ALL)
    list_of_lists.append(acs_list)
nyc_acs=pd.concat([pd.DataFrame(j) for j in list_of_lists])  

In [38]:
nyc_acs.columns=['name', 'total','white_alone', 'black_alone', 'asian_alone', 'med_income', 'state', 'county', 'tract']
nyc_acs['GEOID']=nyc_acs['state']+nyc_acs['county']+nyc_acs['tract']
nyc_acs['percent_white'] = 100* (nyc_acs['white_alone'].astype('int')/nyc_acs['total'].astype('int'))
nyc_acs['percent_black'] = 100* (nyc_acs['black_alone'].astype('int')/nyc_acs['total'].astype('int'))
nyc_acs['percent_asian'] = 100* (nyc_acs['asian_alone'].astype('int')/nyc_acs['total'].astype('int'))

In [39]:
nyc_acs.head()

,name,total,white_alone,black_alone,asian_alone,med_income,state,county,tract,GEOID,percent_white,percent_black,percent_asian
0,"Census Tract 165, New York County, New York",6674.0,5425.0,148.0,469.0,184691.0,36,061,016500,36061016500,81.285586,2.217561,7.027270
1,"Census Tract 166, New York County, New York",6002.0,2824.0,859.0,658.0,47778.0,36,061,016600,36061016600,47.050983,14.311896,10.963012
2,"Census Tract 167, New York County, New York",6058.0,5236.0,36.0,480.0,203711.0,36,061,016700,36061016700,86.431165,0.594256,7.923407
3,"Census Tract 168, New York County, New York",5189.0,1721.0,1915.0,355.0,27222.0,36,061,016800,36061016800,33.166313,36.904991,6.841395
4,"Census Tract 169, New York County, New York",8272.0,6895.0,113.0,537.0,131097.0,36,061,016900,36061016900,83.353482,1.366054,6.491779


***NDVI, NDWI, Paved Surface indices

In [ ]:
#import raster files